## 第4章 「整然データ」を作る
### 4.1 列に(変数ではなく)値が入っているとき
#### 4.1.1 1列に集める

In [ ]:
import pandas as pd
pew = pd.read_csv("../data/pew.csv")
display("最初の5列を表示する", pew.iloc[:, 0:5])
pew_long = pew.melt(id_vars = "religion", var_name = "income", value_name = "count")
display("縦持ちデータを表示する", pew_long)

In [ ]:
import polars as pl
pew = pl.read_csv("../data/pew.csv")
display("最初の5列を表示する", pew[:, 0:5])
pew_long = pew.melt(id_vars = "religion", variable_name = "income", value_name = "count")
display("縦持ちデータを表示する", pew_long)

#### 4.1.2 複数の列を残す

In [ ]:
import pandas as pd
billboard = pd.read_csv("../data/billboard.csv")
display("列および行の先頭部分を表示する", billboard.iloc[0:5, 0:16])
billboard_long = billboard\
    .melt(id_vars = ["year", "artist", "track", "time", "date.entered"],
           var_name = "week",
             value_name = "rating")
display("縦持ちデータを表示する", billboard_long)

In [ ]:
import polars as pl
billboard = pl.read_csv("../data/billboard.csv")
display("列および行の先頭部分を表示する", billboard[0:5, 0:16])
billboard_long = billboard\
    .melt(id_vars = ["year", "artist", "track", "time", "date.entered"],
           variable_name = "week",
             value_name = "rating")
display("縦持ちデータを表示する", billboard_long)

### 4.2 複数の変数を含む列があるとき

In [ ]:
import pandas as pd
ebola = pd.read_csv("../data/country_timeseries.csv")
display("先頭のデータを表示する", ebola.head(n = 3))
ebola_long = ebola.melt(id_vars = ["Date", "Day"])
display("縦持ちデータを表示する", ebola_long.head(n = 3))

In [ ]:
import polars as pl
ebola = pl.read_csv("../data/country_timeseries.csv")
display("先頭のデータを表示する", ebola.head(n = 3))
ebola_long = ebola.melt(id_vars = ["Date", "Day"])
display("縦持ちデータを表示する", ebola_long.head(n = 3))

#### 4.2.1 列を分解して個別に追加する

In [ ]:
import pandas as pd
ebola = pd.read_csv("../data/country_timeseries.csv")
ebola_long = ebola.melt(id_vars = ["Date", "Day"])

variable_split = ebola_long["variable"].str.split("_")
display("発症_国名のデータを分割する", variable_split.head(n = 3))
print("分割した列のデータ型", type(variable_split), "\n")
print("分割した列の最初の要素のデータ型", type(variable_split[0]), "\n")

status_values = variable_split.str.get(0)
country_values = variable_split.str.get(1)
ebola_long["status"] = status_values
ebola_long["country"] = country_values
display("分割した列を縦持ちデータに追加する", ebola_long)

In [ ]:
import polars as pl
ebola = pl.read_csv("../data/country_timeseries.csv")
ebola_long = ebola.melt(id_vars = ["Date", "Day"])

variable_split = ebola_long["variable"].str.split("_")
display("発症_国名のデータを分割する", variable_split.head(n = 3))
print("分割した列のデータ型", type(variable_split), "\n")
print("分割した列の最初の要素のデータ型", type(variable_split[0]), "\n")

df_variable_split = ebola_long["variable"]\
        .str.split_exact("_", 1)\
                .struct.rename_fields(["status", "country"])\
                            .alias("fields")\
                                .to_frame().unnest("fields")

status_values = df_variable_split["status"]
country_values = df_variable_split["country"]
ebola_long = ebola_long.with_columns([
    status_values,
    country_values
])
display("分割した列を縦持ちデータに追加する", ebola_long)

#### 4.2.2 分割と結合を一度に行う

In [ ]:
import pandas as pd
ebola = pd.read_csv("../data/country_timeseries.csv")

ebola_long = ebola.melt(id_vars = ["Date", "Day"])

variable_split = ebola_long.variable.str.split("_", expand = True)
ebola_long[ ["status", "country"] ] = variable_split
display("分割した列を縦持ちデータに追加する", ebola_long.head(n = 3))

In [ ]:
import polars as pl
ebola = pl.read_csv("../data/country_timeseries.csv")

#polarsで同様のことを行う場合 その1
#Series表記の場合、to_fram()とunnest()メソッドが必要
ebola_long = ebola.melt(id_vars = ["Date", "Day"])

ebola_long = ebola_long.with_columns(
    ebola_long["variable"]\
        .str.split_exact("_", 1)\
                .struct.rename_fields(["status", "country"])\
                            .alias("fields")\
                                .to_frame()\
                                        .unnest("fields")
)

display("分割した列を縦持ちデータに追加する", ebola_long.head(n = 3))

In [ ]:
import polars as pl
ebola = pl.read_csv("../data/country_timeseries.csv")

#polarsで同様のことを行う場合 その2
#pl.cpl()を用いる場合、unnestは最後につける
ebola_long = ebola.melt(id_vars = ["Date", "Day"])

ebola_long = ebola_long.with_columns(
    pl.col("variable")\
        .str.split_exact("_", 1)\
                .struct.rename_fields(["status", "country"])\
                            .alias("fields")
).unnest("fields")

display("分割した列を縦持ちデータに追加する", ebola_long.head(n = 3))

### 4.3 行と列の両方に変数があるとき

In [ ]:
import pandas as pd
weather = pd.read_csv("../data/weather.csv")
display("元のデータの表示", weather.head(n = 3))

weather_melt = weather.melt(
    id_vars = ["id", "year", "month", "element"],
    var_name = "day",
    value_name = "temp"
)
display("縦持ちデータの表示", weather_melt.head(n = 3))

weather_tidy = weather_melt.pivot(
    index = ["id", "year", "month", "day"],
    columns = "element",
    values = "temp"
)
display("気温の最大値・最小値をピボット展開したデータの表示", weather_tidy.head(n = 3))

#pandasで層別indexを分解したい場合
#weather_tidy_flat = weather_tidy.reset_index()

In [ ]:
import polars as pl
weather = pl.read_csv("../data/weather.csv")
display("元のデータの表示", weather.head(n = 3))

weather_melt = weather.melt(
    id_vars = ["id", "year", "month", "element"],
    variable_name = "day",
    value_name = "temp"
)
display("縦持ちデータの表示", weather_melt.head(n = 3))

weather_tidy = weather_melt.pivot(
    index = ["id", "year", "month", "day"],
    columns = "element",
    values = "temp"
)
display("気温の最大値・最小値をピボット展開したデータの表示", weather_tidy.head(n = 3))

In [3]:
import pandas as pd
weather = pd.read_csv("../data/weather.csv")
weather_melt = weather.melt(
    id_vars = ["id", "year", "month", "element"],
    var_name = "day",
    value_name = "temp"
)

# 先ほどのmelt, pivotをメソッドチェーンで記述する
# データ処理の方向性が決まったら、処理の一連をチェーンする形で書き直すのが良いだろう
weather_tidy = (
    weather_melt\
        .pivot(index = ["id", "year", "month", "day"],
                     columns = "element",
                     values = "temp")\
                        .reset_index()
)
display("気温の最大値・最小値をピボット展開したデータの表示", weather_tidy.head(n = 3))

'気温の最大値・最小値をピボット展開したデータの表示'

element,id,year,month,day,tmax,tmin
0,MX17004,2010,1,d1,NaN,NaN
1,MX17004,2010,1,d10,NaN,NaN
2,MX17004,2010,1,d11,NaN,NaN


In [4]:
import polars as pl
weather = pl.read_csv("../data/weather.csv")
weather_melt = weather.melt(
    id_vars = ["id", "year", "month", "element"],
    variable_name = "day",
    value_name = "temp"
)

# 先ほどのmelt, pivotをメソッドチェーンで記述する
# データ処理の方向性が決まったら、処理の一連をチェーンする形で書き直すのが良いだろう
weather_tidy = weather.melt(
    id_vars = ["id", "year", "month", "element"],
    variable_name = "day",
    value_name = "temp"
    ).pivot(
        index = ["id", "year", "month", "day"],
        columns = "element",
        values = "temp"
        )
display("気温の最大値・最小値をピボット展開したデータの表示", weather_tidy.head(n = 3))

'気温の最大値・最小値をピボット展開したデータの表示'

id,year,month,day,tmax,tmin
str,i64,i64,str,str,str
"""MX17004""",2010,1,"""d1""","""NA""","""NA"""
"""MX17004""",2010,2,"""d1""","""NA""","""NA"""
"""MX17004""",2010,3,"""d1""","""NA""","""NA"""
